In [1]:
from src.model.system import System
from src.data.datamodule import MyDatamodule
import torch
from tqdm.notebook import tqdm
import numpy as np

In [29]:
system = System.load_from_checkpoint('/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_4/cp_epoch=80_val_loss=1.000_val_acc=0.654.ckpt')
system.freeze()
hparams = system.hparams
dm = MyDatamodule(**hparams)
dm.setup('fit')
ds = dm.val_dataset

In [17]:
def get_emb(system, img):
    with torch.no_grad():
        feature = system.model.forward_features(img.unsqueeze(0).cuda())
        emb = system.model.head.global_pool(feature)
    return emb

def get_embs_from_ds(system, ds):
    embs = []
    for i in tqdm(range(len(ds))):
        embs.append(get_emb(system, ds[i][0]).cpu().numpy())
    return np.concatenate(embs)

In [4]:
checkpoints = [
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_0/cp_epoch=45_val_loss=0.760_val_acc=0.721.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_1/cp_epoch=108_val_loss=0.613_val_acc=0.812.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_2/cp_epoch=65_val_loss=0.605_val_acc=0.756.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_3/cp_epoch=64_val_loss=0.541_val_acc=0.802.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/wr_baseline_4/cp_epoch=51_val_loss=0.666_val_acc=0.758.ckpt'
]
embs = []
for cp in checkpoints:
    system = System.load_from_checkpoint(cp)
    system.freeze()
    hparams = system.hparams
    dm = MyDatamodule(**hparams)
    dm.setup('fit')
    ds = dm.val_dataset
    embs.append(get_embs_from_ds(system, ds))

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

In [5]:
embs = np.concatenate(embs)

In [6]:
np.save('dataset/WR2021/embs.npy',embs)

In [63]:
torch.nn.functional.softmax(system(img.unsqueeze(0).cuda())[0], dim=0)

tensor([0.1270, 0.0720, 0.2451, 0.1583, 0.0067, 0.0195, 0.3715],
       device='cuda:0')

In [2]:
import timm

In [3]:
model = timm.create_model('seresnext101_64x4d', pretrained=True)

In [4]:
model.eval()
dm = MyDatamodule(
    images_path='/home/AZA/PycharmProjects/competitions/cvppa2023/dataset/WR2021/images/', 
    labels_path='/home/AZA/PycharmProjects/competitions/cvppa2023/dataset/WR2021/labels.csv', 
    fold=0, 
    size=(224,224), 
    augmentation_p=False, 
    train_batch_size=4,
    val_batch_size=8, 
    num_workers=8)
dm.setup('fit')
ds = dm.val_dataset + dm.train_dataset

In [5]:
embs = []
with torch.no_grad():
    for i in tqdm(range(len(ds))):
        img = ds[i][0].unsqueeze(0)
        features = model.forward_features(img)
        emb = model.global_pool(features)
        embs.append(emb.cpu().numpy())

  0%|          | 0/1332 [00:00<?, ?it/s]

In [6]:
embs = np.concatenate(embs)

In [8]:
np.save('dataset/WR2021/baseline/embs_se101.npy',embs)

In [9]:
names = ds.datasets[0].labels['names'].tolist() + ds.datasets[1].labels['names'].tolist()

In [10]:
np.save('dataset/WR2021/baseline/names_se101.npy', names)

In [10]:
checkpoints = [
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_0/cp_epoch=65_val_loss=0.856_val_acc=0.668.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_1/cp_epoch=58_val_loss=0.984_val_acc=0.646.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_2/cp_epoch=50_val_loss=1.180_val_acc=0.620.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_3/cp_epoch=42_val_loss=0.910_val_acc=0.614.ckpt',
    '/mnt/IMAGING/HUB/MODELS/CHECKPOINTS/cvppa/cvppa/baseline_4/cp_epoch=80_val_loss=1.000_val_acc=0.654.ckpt'
]
probs = []
labels = []
names = []
for cp in checkpoints:
    system = System.load_from_checkpoint(cp)
    system.freeze()
    hparams = system.hparams
    dm = MyDatamodule(**hparams)
    dm.setup('fit')
    ds = dm.val_dataset
    names.extend(ds.labels['names'].tolist())
    with torch.no_grad():
        for i in tqdm(range(len(ds))):
            img, label = ds[i]
            pred = system(img.unsqueeze(0).cuda())
            labels.append(label)
            probs.append(torch.nn.functional.softmax(system(img.unsqueeze(0).cuda())[0], dim=0).cpu().numpy())

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/267 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

In [11]:
probs = np.stack(probs)

In [12]:
np.save('dataset/WR2021/probs.npy', probs)

In [13]:
np.save('dataset/WR2021/labels.npy', labels)

In [14]:
np.save('dataset/WR2021/names.npy', names)